# db_mover.ipynb
This Jupyter Notebook will copy the PostgreSQL DB co2emission_db and all its tables to a local sqlite DB called fossil_fuel_db.sqlite

In [25]:
# Import Dependencies
import pandas as pd
import sqlalchemy
import psycopg2
from postgres_key import DB_USER, DB_KEY, DB_NAME

def open_connection():
    conn = psycopg2.connect(host="localhost", port=5432,
                            database=DB_NAME, user=DB_USER, password=DB_KEY)
    return conn
conn = open_connection()

In [27]:
target_engine = sqlalchemy.create_engine('sqlite:///fossil_fuel_db.sqlite')

# Establish Connections
target_conn = target_engine.connect()

# Using pandas to read data out of SQL
my_data_df = pd.read_sql("SELECT * FROM co2_emission", conn)
my_data_df

,country_id,country,year,year_range,fuel_type,emission_value
0,12,Algeria,1965,1965-1975,co2_mtco2,5.568753
1,12,Algeria,1966,1965-1975,co2_mtco2,6.867506
2,12,Algeria,1967,1965-1975,co2_mtco2,6.369307
3,12,Algeria,1968,1965-1975,co2_mtco2,6.771137
4,12,Algeria,1969,1965-1975,co2_mtco2,7.592118
...,...,...,...,...,...,...
4158,704,Vietnam,2016,2015-2020,co2_mtco2,198.082509
4159,704,Vietnam,2017,2015-2020,co2_mtco2,199.489386
4160,704,Vietnam,2018,2015-2020,co2_mtco2,240.981486
4161,704,Vietnam,2019,2015-2020,co2_mtco2,292.073219


In [28]:
cursor = conn.cursor()
cursor.execute("""SELECT table_name FROM information_schema.tables
       WHERE table_schema = 'public'""")
table_names = [table[0] for table in cursor.fetchall()] 
for table in table_names:
    my_data_df = pd.read_sql(f"SELECT * FROM {table}", conn)
    my_data_df.to_sql(table, target_conn, if_exists="replace")
    print(table)
target_engine.dispose()

country_master
fuel_consumption
fuel_production
co2_emission
gdp


In [29]:
target_engine = sqlalchemy.create_engine('sqlite:///fossil_fuel_db.sqlite')
target_conn = target_engine.connect()

,index,country_id,country,region
0,0,12,Algeria,Africa
1,1,24,Angola,Africa
2,2,32,Argentina,S. & Cent. America
3,3,36,Australia,Asia Pacific
4,4,31,Azerbaijan,CIS
...,...,...,...,...
94,94,68,Bolivia,S. & Cent. America
95,95,104,Myanmar,Asia Pacific
96,96,496,Mongolia,Asia Pacific
97,97,688,Serbia,Europe


In [31]:
test_df = pd.read_sql(f"SELECT * FROM {table_names[2]}", target_conn)
test_df

,index,country_id,country,year,year_range,fuel_type,production_value
0,0,12,Algeria,1965,1965-1975,oilprod_ej,1.273141
1,1,12,Algeria,1966,1965-1975,oilprod_ej,1.638700
2,2,12,Algeria,1967,1965-1975,oilprod_ej,1.882344
3,3,12,Algeria,1968,1965-1975,oilprod_ej,2.060670
4,4,12,Algeria,1969,1965-1975,oilprod_ej,2.165293
...,...,...,...,...,...,...,...
7056,7056,840,US,2016,2015-2020,biodiesel_prod_ej,0.195438
7057,7057,840,US,2017,2015-2020,biodiesel_prod_ej,0.198955
7058,7058,840,US,2018,2015-2020,biodiesel_prod_ej,0.231544
7059,7059,840,US,2019,2015-2020,biodiesel_prod_ej,0.214924
